In [2]:
import sklearn
import pandas as pd

In [3]:
df = pd.read_parquet("/beegfs/ws/0/s4610340-energy_behavior/yahor/kaggle-predict_energy_behavior_of_prosumers/data/processed/train/make_features/df_train_features.parquet")
df

,county,is_business,product_type,target,is_consumption,row_id,date,lowest_price_per_mwh,highest_price_per_mwh,eic_count,...,target_7,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour)
0,0,0,1,0.713000,0,0,2021-09-01,NaN,NaN,NaN,...,NaN,1,3,9,2021,0_0_1_0,-0.866025,-0.500000,0.000000,1.000000
1,0,0,1,96.589996,1,1,2021-09-01,NaN,NaN,NaN,...,NaN,1,3,9,2021,0_0_1_1,-0.866025,-0.500000,0.000000,1.000000
2,0,0,2,0.000000,0,2,2021-09-01,NaN,NaN,NaN,...,NaN,1,3,9,2021,0_0_2_0,-0.866025,-0.500000,0.000000,1.000000
3,0,0,2,17.313999,1,3,2021-09-01,NaN,NaN,NaN,...,NaN,1,3,9,2021,0_0_2_1,-0.866025,-0.500000,0.000000,1.000000
4,0,0,3,2.904000,0,4,2021-09-01,NaN,NaN,NaN,...,NaN,1,3,9,2021,0_0_3_0,-0.866025,-0.500000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017819,15,1,0,197.233002,1,2018347,2023-05-31,29.0,34.0,15.0,...,625.776001,31,3,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926
2017820,15,1,1,0.000000,0,2018348,2023-05-31,29.0,34.0,20.0,...,0.000000,31,3,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926
2017821,15,1,1,28.403999,1,2018349,2023-05-31,29.0,34.0,20.0,...,625.776001,31,3,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926
2017822,15,1,3,0.000000,0,2018350,2023-05-31,29.0,34.0,55.0,...,0.000000,31,3,5,2023,15_1_3_0,0.522133,-0.852864,-0.258819,0.965926


In [4]:
for col in df.columns:
    print(col)

county
is_business
product_type
target
is_consumption
row_id
date
lowest_price_per_mwh
highest_price_per_mwh
eic_count
installed_capacity
euros_per_mwh
hours_ahead
temperature
dewpoint
cloudcover_high
cloudcover_low
cloudcover_mid
cloudcover_total
10_metre_u_wind_component
10_metre_v_wind_component
direct_solar_radiation
surface_solar_radiation_downwards
snowfall
total_precipitation
hours_ahead_fl
temperature_fl
dewpoint_fl
cloudcover_high_fl
cloudcover_low_fl
cloudcover_mid_fl
cloudcover_total_fl
10_metre_u_wind_component_fl
10_metre_v_wind_component_fl
direct_solar_radiation_fl
surface_solar_radiation_downwards_fl
snowfall_fl
total_precipitation_fl
hours_ahead_fd_7d
temperature_fd_7d
dewpoint_fd_7d
cloudcover_high_fd_7d
cloudcover_low_fd_7d
cloudcover_mid_fd_7d
cloudcover_total_fd_7d
10_metre_u_wind_component_fd_7d
10_metre_v_wind_component_fd_7d
direct_solar_radiation_fd_7d
surface_solar_radiation_downwards_fd_7d
snowfall_fd_7d
total_precipitation_fd_7d
hours_ahead_fl_7d
temperature

In [5]:
def train_val_split_temporal(df: pd.DataFrame, val_len: int) -> tuple[pd.DataFrame, pd.DataFrame]:
    train_series: list[pd.DataFrame] = []
    val_series: list[pd.DataFrame] = []
    val_len = pd.Timedelta(days=val_len)

    for segment, df_segment in df.groupby("segment"):
        date_segment_start = df_segment["date"].iloc[0]
        date_segment_end = df_segment["date"].iloc[-1]
        if (date_segment_end - date_segment_start) >= 2*val_len:
            val_start = date_segment_end - val_len
            train_series.append(df_segment.loc[df_segment["date"] < val_start])
            val_series.append(df_segment.loc[df_segment["date"] >= val_start])
        else:
            train_series.append(df_segment)

    return pd.concat(train_series), pd.concat(val_series)

df_train, df_val = train_val_split_temporal(df, val_len=100)
print(len(df_val["segment"].unique()))
print(len(df_train["segment"].unique()))

138
138


In [6]:
X_train = df_train.drop("target", axis=1)
y_train = df_train["target"]

X_val = df_val.drop("target", axis=1)
y_val = df_val["target"]

### Constant regressor

#### Constant regressor - all 0

In [7]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error

model_all_zeros = DummyRegressor(strategy="constant", constant=0)

model_all_zeros.fit(X=X_train, y=y_train)
predictions_val = model_all_zeros.predict(X_val)

mean_absolute_error(y_val, predictions_val)


338.9043739150225

#### Constant regressor - median

In [8]:
model_all_median = DummyRegressor(strategy="median")

model_all_median.fit(X=X_train, y=y_train)
predictions_val = model_all_median.predict(X_val)

mean_absolute_error(y_val, predictions_val)

329.79633

#### Constant regressor - mean

In [9]:
model = DummyRegressor(strategy="mean")

model.fit(X=X_train, y=y_train)
predictions_val = model.predict(X_val)

mean_absolute_error(y_val, predictions_val)

395.4935

### Rule-based (Tree)

In [10]:
from sklearn.tree import DecisionTreeRegressor

tree_features = ["is_business", "product_type"]

model_consumption = DecisionTreeRegressor(
    criterion='absolute_error',
    #splitter="best",
    #min_samples_leaf=10,
    max_depth=2
)

model_production = DecisionTreeRegressor(
    criterion='absolute_error',
    #splitter="best",
    #min_samples_leaf=10,
    max_depth=2
)

mask_train = df_train["is_consumption"] == 1
X_train = df_train.loc[mask_train].drop(["target"], axis=1)[tree_features]
y_train = df_train.loc[mask_train]["target"]

mask_val = df_val["is_consumption"] == 1
X_val = df_val.loc[mask_val].drop(["target"], axis=1)[tree_features]
y_val = df_val.loc[mask_val]["target"]

print("Training consumption model ...")
model_consumption.fit(X=X_train, y=y_train)
predictions_val = model_consumption.predict(X_val)

print(mean_absolute_error(y_val, predictions_val))

mask_train = df_train["is_consumption"] == 0
X_train = df_train.loc[mask_train].drop(["target"], axis=1)[tree_features]
y_train = df_train.loc[mask_train]["target"]

mask_val = df_val["is_consumption"] == 0
X_val = df_val.loc[mask_val].drop(["target"], axis=1)[tree_features]
y_val = df_val.loc[mask_val]["target"]

print("Training production model ...")
model_production.fit(X=X_train, y=y_train)
predictions_val = model_production.predict(X_val)

print(mean_absolute_error(y_val, predictions_val))



Training consumption model ...
